In [1]:
import pygame as pg
from random import choice
import time
import random
import sys
import heapq

pygame 2.6.1 (SDL 2.28.4, Python 3.13.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Cell:
    def __init__(self, x, y, thick, color='black'):
        self.x, self.y, self.thick = x, y, thick
        self.wall = {'top': True, 'right': True, 'left': True, 'bottom': True}
        self.visited = False
        self.color = color
        self.current_color = 'white'
        self.base_color = 'white'
        self.action = 'simple'

    def draw(self, sc, tile):
        x, y = self.x * tile, self.y * tile
        pg.draw.rect(sc, self._bg(), (x, y, tile, tile))
        if self.wall['top']:
            pg.draw.line(sc, pg.Color(self.color), (x, y), (x + tile, y), self.thick)
        if self.wall['right']:
            pg.draw.line(sc, pg.Color(self.color), (x + tile, y), (x + tile, y + tile), self.thick)
        if self.wall['left']:
            pg.draw.line(sc, pg.Color(self.color), (x, y), (x, y + tile), self.thick)
        if self.wall['bottom']:
            pg.draw.line(sc, pg.Color(self.color), (x, y + tile), (x + tile, y + tile), self.thick)

    def check_cell(self, x, y, cols, rows, grid_cells):
        find_index = lambda x, y: x + y * cols
        if x < 0 or x > cols - 1 or y < 0 or y > rows - 1:
            return False
        return grid_cells[find_index(x, y)]

    def check_neighbors(self, cols, rows, grid_cells):
        neighbors = []
        top = self.check_cell(self.x, self.y - 1, cols, rows, grid_cells)
        right = self.check_cell(self.x + 1, self.y, cols, rows, grid_cells)
        left = self.check_cell(self.x - 1, self.y, cols, rows, grid_cells)
        bottom = self.check_cell(self.x, self.y + 1, cols, rows, grid_cells)
        if top and not top.visited:
            neighbors.append(top)
        if right and not right.visited:
            neighbors.append(right)
        if left and not left.visited:
            neighbors.append(left)
        if bottom and not bottom.visited:
            neighbors.append(bottom)
        return choice(neighbors) if neighbors else False

    def set_color(self, new_color):
        self.current_color = new_color

    def reset_color(self):
        self.current_color = self.base_color

    def set_base(self, color):
        self.base_color = color
        self.current_color = color

    def _bg(self):
        return self.current_color

    def set_action(self, action):
        self.action = action
        if action == 'fast_speed':
            self.set_base('darkblue')
        elif action == 'nothing':
            self.set_base('white')
        elif action == 'slow_speed':
            self.set_base('darkmagenta')
        elif action == 'add_health':
            self.set_base('firebrick4')
        elif action == 'go_spawn':
            self.set_base('lime')
        elif action == 'teleport':
            self.set_base('lightslateblue')

In [3]:
class Maze:
    def __init__(self, cols, rows):
        self.cols, self.rows = cols, rows
        self.thick = 4
        self.grid_cells = [Cell(col, row, self.thick) for row in range(self.rows) for col in range(self.cols)]
        self.sp_cnt = 0
        self.sl_cnt = 0
        self.h_cnt = 0
        self.s_cnt = 0
        self.tp = 0

    def remove_walls(self, cur, next):
        dx = cur.x - next.x
        if dx == 1:
            cur.wall['left'] = False
            next.wall['right'] = False
        elif dx == -1:
            cur.wall['right'] = False
            next.wall['left'] = False
        dy = cur.y - next.y
        if dy == 1:
            cur.wall['top'] = False
            next.wall['bottom'] = False
        elif dy == -1:
            cur.wall['bottom'] = False
            next.wall['top'] = False

    def generate_maze(self):
        cur_cell = self.grid_cells[0]
        arr = []
        break_cnt = 1
        while break_cnt != len(self.grid_cells):
            cur_cell.visited = True
            next_cell = cur_cell.check_neighbors(self.cols, self.rows, self.grid_cells)
            if next_cell:
                next_cell.visited = True
                break_cnt += 1
                arr.append(cur_cell)
                self.remove_walls(cur_cell, next_cell)
                cur_cell = next_cell
            elif arr:
                cur_cell = arr.pop()
        for i in self.get_walls():
            if random.random() < 0.3:
                self.remove_wall(i)
        for cell in self.grid_cells:
            if random.random() < 0.05:
                cell.set_base('darkgreen')
            else:
                if random.random() < 0.005 and self.sp_cnt < 1:
                    cell.set_action('fast_speed')
                    self.sp_cnt += 1
                elif random.random() < 0.005 and self.sl_cnt < 1:
                    cell.set_action('slow_speed')
                    self.sl_cnt += 1
                elif random.random() < 0.009 and self.h_cnt < 2:
                    cell.set_action('add_health')
                    self.h_cnt += 1
                elif random.random() < 0.005 and self.s_cnt < 1:
                    cell.set_action('go_spawn')
                    self.s_cnt += 1
                elif random.random() < 0.009 and self.tp < 2:
                    self.tp += 1
                    cell.set_action('teleport')
        return self.grid_cells

    def get_walls(self):
        walls = []
        for cell in self.grid_cells:
            if cell.x > 1 and cell.wall['left']:
                walls.append((cell, 'left'))
            if cell.x < self.cols - 2 and cell.wall['right']:
                walls.append((cell, 'right'))
            if cell.y > 1 and cell.wall['top']:
                walls.append((cell, 'top'))
            if cell.y < self.rows - 2 and cell.wall['bottom']:
                walls.append((cell, 'bottom'))
        return walls

    def get_cell(self, x, y):
        for cell in self.grid_cells:
            if cell.x == x and cell.y == y:
                return cell

    def remove_wall(self, wall):
        direct = wall[1]
        cell = wall[0]
        if direct == 'left':
            cell2 = self.get_cell(cell.x - 1, cell.y)
            cell.wall[direct] = False
            cell2.wall['right'] = False
        elif direct == 'right':
            cell2 = self.get_cell(cell.x + 1, cell.y)
            cell.wall[direct] = False
            cell2.wall['left'] = False
        elif direct == 'top':
            cell2 = self.get_cell(cell.x, cell.y - 1)
            cell.wall[direct] = False
            cell2.wall['bottom'] = False
        elif direct == 'bottom':
            cell2 = self.get_cell(cell.x, cell.y + 1)
            cell.wall[direct] = False
            cell2.wall['top'] = False

In [4]:
class Player:
    def __init__(self, x, y, color = (250, 120, 60)):
        self.x = int(x)
        self.y = int(y)
        self.size = 10
        self.rect = pg.Rect(self.x, self.y, self.size, self.size)
        self.color = color
        self.velX = 0
        self.velY = 0
        self.left_press = False
        self.right_press = False
        self.up_press = False
        self.down_press = False
        self.speed = 2
        self.lives = 5

    def get_current_cell(self, x, y, grid_cells):
        for cell in grid_cells:
            if cell.x == x and cell.y == y:
                return cell

    def check_move(self, tile, grid_cells, thick, cols, rows):
        cur_cell_x, cur_cell_y = self.x // tile, self.y // tile
        if cur_cell_x < 0 or cur_cell_x >= cols or cur_cell_y < 0 or cur_cell_y >= rows:
            return 
        cur_cell = self.get_current_cell(cur_cell_x, cur_cell_y, grid_cells)
        if not cur_cell:
            return
        cur_cell_abs_x, cur_cell_abs_y = cur_cell_x * tile, cur_cell_y * tile
        if self.left_press:
            if cur_cell.wall['left']:
                if self.x <= cur_cell_abs_x + thick:
                    self.left_press = False
                    self.velX = 0
        if self.right_press:
            if cur_cell.wall['right']:
                if self.x >= cur_cell_abs_x + tile - thick - self.size:
                    self.right_press = False
                    self.velX = 0
        if self.up_press:
            if cur_cell.wall['top']:
                if self.y <= cur_cell_abs_y + thick:
                    self.up_press = False
                    self.velY = 0
        if self.down_press:
            if cur_cell.wall['bottom']:
                if self.y >= cur_cell_abs_y + tile - thick - self.size:
                    self.down_press = False
                    self.velY = 0

    def draw(self, screen):
        pg.draw.rect(screen, self.color, self.rect)

    def update(self, tile, grid_cells, thick, cols, rows):
        new_x, new_y = self.x, self.y
        if self.left_press:
            new_x -= self.speed
        if self.right_press:
            new_x += self.speed
        if self.up_press:
            new_y -= self.speed
        if self.down_press:
            new_y += self.speed
        if self._can_(new_x, new_y, tile, grid_cells, thick, cols, rows):
            self.x = new_x
            self.y = new_y
        self.rect = pg.Rect(int(self.x), int(self.y), self.size, self.size)

    def _can_(self, new_x, new_y, tile, grid_cells, thick, cols, rows):

        if new_x < 0 or new_x + self.size > cols * tile or new_y < 0 or new_y + self.size > rows * tile:
            return False
        cur_cell_x = new_x // tile
        cur_cell_y = new_y // tile
        if cur_cell_x < 0 or cur_cell_x >= cols or cur_cell_y < 0 or cur_cell_y >= rows:
            return False
        cur_cell = self.get_current_cell(cur_cell_x, cur_cell_y, grid_cells)
        if not cur_cell:
            return False
        cur_cell_abs_x = cur_cell_x * tile
        cur_cell_abs_y = cur_cell_y * tile
        if self.left_press and cur_cell.wall['left'] and self.x <= cur_cell_abs_x + thick:
            return False
        if self.right_press and cur_cell.wall['right'] and self.x + self.size >= cur_cell_abs_x + tile - thick:
            return False
        if self.up_press and cur_cell.wall['top'] and self.y <= cur_cell_abs_y + thick:
            return False
        if self.down_press and cur_cell.wall['bottom'] and self.y + self.size >= cur_cell_abs_y + tile - thick:
            return False
        return True

    def change_speed(self, speed):
        self.speed = speed

    def change_lives(self, ch):
        self.lives += ch

In [5]:
class Game:
    def __init__(self, goal_cell, tile):
        self.font = pg.font.SysFont('impact', 35)
        self.message_color = pg.Color('darkorange')
        self.goal_cell = goal_cell
        self.tile = tile

    def add_goal_point(self, screen):
        img_path = 'a58ae648c4eef0537c14cece8ce272f5.jpg'
        img = pg.image.load(img_path)
        img = pg.transform.scale(img, (self.tile, self.tile))
        screen.blit(img, (self.goal_cell.x * self.tile, self.goal_cell.y * self.tile))

    def message(self):
        msg = self.font.render('You Win!!', True, self.message_color)
        return msg

    def lose(self):
        msg = self.font.render('You LOSE!!', True, self.message_color)
        return msg

    def is_game_over(self, player, exit, lives):
        if lives < 1:
            return True
        if exit:
            goal_cell_abs_x, goal_cell_abs_y = self.goal_cell.x * self.tile, self.goal_cell.y * self.tile
            if player.x >= goal_cell_abs_x and player.y >= goal_cell_abs_y:
                return True
        return False

In [6]:
class Clock:
    def __init__(self):
        self.start_time = None
        self.elapsed_time = 0
        self.font = pg.font.SysFont("monospace", 35)
        self.message_color = pg.Color("yellow")

    def start_timer(self):
        self.start_time = time.time()

    def update_timer(self):
        if self.start_time is not None:
            self.elapsed_time = time.time() - self.start_time

    def display_timer(self):
        secs = int(self.elapsed_time % 60)
        mins = int(self.elapsed_time / 60)
        my_time = self.font.render(f"{mins:02}:{secs:02}", True, self.message_color)
        return my_time

    def stop_timer(self):
        self.start_time = None

In [17]:
class Bot_easy:
    def __init__(self, x, y, tile, color='cyan'):
        self.x = x
        self.y = y
        self.st = (self.x, self.y)
        self.size = 10
        self.rect = pg.Rect(self.x, self.y, self.size, self.size)
        self.color = color
        self.speed = 2
        self.path = []
        self.tile = tile

    def find(self, maze, player):
        st = (self.x // self.tile, self.y // self.tile)
        end = (player.x // self.tile, player.y // self.tile)
        self.path = self._A_(maze.grid_cells, maze.cols, maze.rows, st, end)

    def follow(self, sc):
       if self.path:
           for (x, y) in self.path:
               pg.draw.circle(sc, pg.Color('red'), (x * self.tile + self.tile // 2, y * self.tile + self.tile // 2), 5)
           x, y = self.path.pop(0)
           self.x = x * self.tile + self.tile // 2 - self.size // 2
           self.y = y * self.tile + self.tile // 2 - self.size // 2
           self.rect.topleft = (self.x, self.y)

    def draw(self, sc):
        pg.draw.rect(sc, self.color, self.rect)

    @staticmethod
    def _hev_(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    @staticmethod
    def _A_(grid_cells, cols, rows, st, end):
        if st == end:
            return []
        start_index = st[0] + st[1] * cols
        end_index = end[0] + end[1] * cols
        heap = [(0, start_index, [])]
        visited = set()
        while heap:
            cost, current_index, path = heapq.heappop(heap)
            if current_index == end_index:
                return [(grid_cells[i].x, grid_cells[i].y) for i in path] + [(grid_cells[end_index].x, grid_cells[end_index].y)]
            if current_index in visited:
                continue
            visited.add(current_index)
            x, y = grid_cells[current_index].x, grid_cells[current_index].y
            neighbors = []
            if y > 0 and not grid_cells[current_index].wall['top']:
                neighbors.append(current_index - cols)
            if x < cols - 1 and not grid_cells[current_index].wall['right']:
                neighbors.append(current_index + 1)
            if y < rows - 1 and not grid_cells[current_index].wall['bottom']:
                neighbors.append(current_index + cols)
            if x > 0 and not grid_cells[current_index].wall['left']:
                neighbors.append(current_index - 1)
            for neighbor_index in neighbors:
                if neighbor_index in visited:
                    continue
                new_cost = cost + 1 + Bot_easy._hev_((grid_cells[neighbor_index].x, grid_cells[neighbor_index].y), end)
                heapq.heappush(heap, (new_cost, neighbor_index, path + [current_index]))
        return []
        
    def show(self):
        pass

In [14]:
class Main():
    def __init__(self, screen, tile=30, cnt = 0):
        self.screen = screen
        self.font = pg.font.SysFont("impact", 30)
        self.message_color = pg.Color("cyan")
        self.running = True
        self.game_over = False
        self.FPS = pg.time.Clock()
        self.cnt = cnt
        self.all = 0
        self.first = True
        self.exit = False
        self.path = False
        self.in_menu = True

    @staticmethod
    def touch(plx, ply, botx, boty):
        dx = abs(plx - botx)
        dy = abs(ply - boty)
        return (dx ** 2 + dy ** 2) ** 0.5 < 15

    def draw_darkness(self, screen, player, radius, cols, rows, tile):
        darkness = pg.Surface((cols * tile, rows * tile), pg.SRCALPHA)
        darkness.fill((0, 0, 0, 255)) 
        light_x = int(player.x + player.size / 2)
        light_y = int(player.y + player.size / 2)
        for r in range(radius, 0, -5):
            alpha = int(255 * (r / radius))
            pg.draw.circle(darkness, (0, 0, 0, alpha), (light_x, light_y), r)
        screen.blit(darkness, (0, 0))
    
    def _draw(self, maze, tile, player, game, clock, cols, rows, bot_easy):
        now = self.first
        for cell in maze.grid_cells:
            if (cell.x, cell.y) == (player.x//tile, player.y//tile):
                if cell.current_color == 'darkgreen':
                    cell.set_base('white')
                    self.cnt += 1
                cell.set_color('gold')
                cnt = self.font.render(f'{self.cnt} из {self.all}', True, self.message_color)
                self.screen.blit(cnt,(610,300))
                if cell.action == 'fast_speed':
                    player.change_speed(player.speed + 2)
                    cell.set_action('nothing')
                if cell.action == 'slow_speed':
                    player.change_speed(player.speed - 1)
                    cell.set_action('nothing')
                if cell.action == 'add_health':
                    if player.lives < 5:
                        player.change_lives(1)
                        cell.set_action('nothing')
                if cell.action == 'go_spawn':
                    bot_easy.x = bot_easy.st[0]
                    bot_easy.y = bot_easy.st[1]
                    bot_easy.path = []
                if cell.action == 'teleport':
                    cells = [cell for cell in maze.grid_cells if cell.current_color != 'darkgreen']
                    new = random.choice(cells)
                    player.x = new.x * tile
                    player.y = new.y * tile
                if self.cnt == self.all and not now:
                    self.exit = True
                if self.touch(player.x, player.y, bot_easy.x, bot_easy.y):
                    player.change_lives(-1)
                    bot_easy.x = bot_easy.st[0]
                    bot_easy.y = bot_easy.st[1]
                    bot_easy.path = []
                live = self.font.render(f'{player.lives} из 5 lives', True, self.message_color)
                self.screen.blit(live,(610,400))
            else:
                cell.reset_color()
            cell.draw(self.screen, tile)
            if cell.current_color == 'darkgreen' and now:
                self.all += 1
        self.first = False
        game.add_goal_point(self.screen)
        player.draw(self.screen)
        bot_easy.draw(self.screen)
        if bot_easy.path and self.path:
            for (x, y) in bot_easy.path:
                pg.draw.circle(self.screen, pg.Color('red'),
                           (x * bot_easy.tile + bot_easy.tile // 2, y * bot_easy.tile + bot_easy.tile // 2),
                           5)
        player.update(tile, maze.grid_cells, 4, cols, rows)
        self.draw_darkness(screen, player, 250, cols, rows, tile)
        if self.game_over:
            clock.stop_timer()
            if player.lives < 1:
                self.screen.blit(game.lose(),(610,120))
            else:
                self.screen.blit(game.message(),(610,120))
        else:
            clock.update_timer()
        self.screen.blit(clock.display_timer(), (625,200))
        pg.display.flip()

    def draw_menu(self):
        self.screen.fill(pg.Color("darkslategray"))
        screen_width = self.screen.get_width()
        screen_height = self.screen.get_height()
        title = self.font.render("Maze Game", True, self.message_color)
        self.screen.blit(title, (screen_width // 2 - title.get_width() // 2, 100))
        rules = [
            "Правила игры:",
            "1. Управляйте помощью стрелок.",
            "2. Собирайте зеленые квадраты, чтобы открыть выход.",
            "3. Избегайте ботов, которые отнимают у вас жизни.",
            "4. Стремитесь к выходу!"
        ]
        for i, line in enumerate(rules):
            rule_text = self.font.render(line, True, self.message_color)
            self.screen.blit(rule_text, (screen_width // 2 - rule_text.get_width() // 2, 200 + 40 * i))
        start_button = pg.Rect(screen_width // 2 - 75, 400, 150, 50)
        pg.draw.rect(self.screen, pg.Color("green"), start_button)
        start_text = self.font.render("Start", True, pg.Color("white"))
        self.screen.blit(start_text, (start_button.centerx - start_text.get_width() // 2,
                                      start_button.centery - start_text.get_height() // 2))
        for event in pg.event.get():
            if event.type == pg.QUIT:
                pg.quit()
                sys.exit()
            if event.type == pg.MOUSEBUTTONDOWN:
                if start_button.collidepoint(event.pos):
                    self.in_menu = False
    
    def main(self, frame_size, tile):
        cols, rows = frame_size[0] // tile, frame_size[-1] // tile
        maze = Maze(cols, rows)
        game = Game(maze.grid_cells[-1], tile)
        player = Player(tile // 3, tile // 3)
        bot_easy = Bot_easy(tile * (cols - 1), 0, tile)
        clock = Clock()
        maze.generate_maze()
        clock.start_timer()
        while self.running:
            if self.in_menu:
                self.draw_menu()
            else:
                self.screen.fill("gray")
                self.screen.fill( pg.Color("darkslategray"), (603, 0, 752, 752))
                for event in pg.event.get():
                    if event.type == pg.QUIT:
                        pg.quit()
                        sys.exit()
                    if event.type == pg.KEYDOWN:
                        if not self.game_over:
                            if event.key == pg.K_p:
                                if self.path:
                                    self.path = False
                                else:
                                    self.path = True
                            if event.key == pg.K_LEFT:
                                player.left_press = True
                            if event.key == pg.K_RIGHT:
                                player.right_press = True
                            if event.key == pg.K_UP:
                                player.up_press = True
                            if event.key == pg.K_DOWN:
                                player.down_press = True
                            player.check_move(tile, maze.grid_cells, maze.thick, cols, rows)
                    if event.type == pg.KEYUP:
                        if not self.game_over:
                            if event.key == pg.K_LEFT:
                                player.left_press = False
                            if event.key == pg.K_RIGHT:
                                player.right_press = False
                            if event.key == pg.K_UP:
                                player.up_press = False
                            if event.key == pg.K_DOWN:
                                player.down_press = False
                            player.check_move(tile, maze.grid_cells, maze.thick, cols, rows)
                player.update(tile, maze.grid_cells, maze.thick, cols, rows)
                if game.is_game_over(player, self.exit, player.lives):
                    self.game_over = True
                    player.left_press = False
                    player.right_press = False
                    player.up_press = False
                    player.down_press = False
                if pg.time.get_ticks() % 15 == 0:
                    bot_easy.find(maze, player)
                if pg.time.get_ticks() % 8 == 0:
                    bot_easy.follow(self.screen)
                self._draw(maze, tile, player, game, clock, cols, rows, bot_easy)
                self.FPS.tick(60)
            pg.display.flip()

In [16]:
pg.init()
pg.font.init()
if __name__ == "__main__":
    window_size = (602, 602)
    screen = (window_size[0] + 150, window_size[-1])
    tile_size = 30
    screen = pg.display.set_mode(screen)
    pg.display.set_caption("Maze")

    game = Main(screen)
    game.main(window_size, tile_size)

SystemExit: 